
# 📒 Kraken Training — **ALTO-only** (Colab, auto-flag detection, GPU/CPU)

This notebook:
- Assumes **ALTO** ground truth (eScriptorium exports)
- Upload ZIPs → auto-extract
- Rebuilds `train.txt` / `val.txt`
- **Auto-detects** Kraken CLI flags (older vs newer): `--lr` vs `--lrate`, `--validation` vs `--evaluation-files`
- Uses Python `subprocess.run` (prints logs live)
- Auto-selects GPU if available

> In Colab: **Runtime → Change runtime type → GPU** first.


In [ ]:

# 0) GPU check
!nvidia-smi || true


## 1) Install Kraken + deps (Py 3.12 / Colab-safe)

In [ ]:

import sys, subprocess
def pip_install(*pkgs):
    print("pip install", " ".join(pkgs))
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", *pkgs])
# Colab-friendly pin (CLI may still be older depending on environment; we'll detect flags later)
pip_install("kraken==5.3.0", "torch>=2.1,<3", "cairocffi", "opencv-python", "lxml", "h5py")


In [ ]:

from importlib.metadata import version, PackageNotFoundError
import shutil, torch, sys, subprocess

def pkg_ver(name):
    try: return version(name)
    except PackageNotFoundError: return "not installed"

print("python:", sys.version.split()[0])
print("kraken:", pkg_ver("kraken"))
print("torch:", torch.__version__)
print("cuda available:", torch.cuda.is_available())
print("ketos path:", shutil.which("ketos"))

# Show a snippet of help to confirm CLI is reachable
subprocess.run(["ketos", "train", "--help"], check=False)


## 2) Config — ALTO fixed, validation split

In [ ]:

# ======= CONFIG (ALTO) =======
FORMAT = "alto"     # hard-coded
VAL_FRACTION = 0.10 # validation split
# =============================


## 3) Upload ALTO ZIP(s) — auto-extract under /content/data_alto

In [ ]:

import os, io, zipfile, re
from google.colab import files

LOCAL_BASE = "/content/data_alto"
os.makedirs(LOCAL_BASE, exist_ok=True)

print("Upload ALTO export ZIP(s). They will be extracted into", LOCAL_BASE)
uploaded = files.upload()

EXTRACTED_ROOTS = []
for name, data in uploaded.items():
    path = os.path.join(LOCAL_BASE, name)
    with open(path, "wb") as f: f.write(data)
    if name.lower().endswith(".zip"):
        base = re.sub(r"\s*\(\d+\)\s*$", "", os.path.splitext(name)[0])
        target = os.path.join(LOCAL_BASE, base)
        os.makedirs(target, exist_ok=True)
        with zipfile.ZipFile(io.BytesIO(data)) as zf:
            zf.extractall(target)
        os.remove(path)
        EXTRACTED_ROOTS.append(target)
    else:
        EXTRACTED_ROOTS.append(LOCAL_BASE)

print("Extracted roots:")
for r in EXTRACTED_ROOTS: print(" -", r if os.path.exists(r) else r+" (missing)")


## 4) Build ALTO XML list + write train/val files

In [ ]:

import os, glob, random, pathlib, xml.etree.ElementTree as ET

# Gather roots
ROOTS = [r for r in EXTRACTED_ROOTS if r and os.path.exists(r)]
assert ROOTS, "No roots found. Upload a ZIP first and re-run."

def is_alto_xml(path):
    try:
        r = ET.parse(path).getroot()
        return isinstance(r.tag, str) and ("alto" in r.tag.lower())
    except Exception:
        return False

# Find XMLs (case-insensitive) and keep only ALTO
xmls = []
for root in ROOTS:
    cands = glob.glob(os.path.join(root, "**", "*.xml"), recursive=True) +             glob.glob(os.path.join(root, "**", "*.XML"), recursive=True)
    altos = [p for p in cands if is_alto_xml(p)]
    print(f"[ALTO] {len(altos):>5} XMLs in {root}")
    xmls.extend(altos)

print("TOTAL ALTO XML files:", len(xmls))
assert len(xmls) >= 3, "Need at least a few ALTO XML pages."

# Train/val split
random.seed(42)
random.shuffle(xmls)
n_val = max(1, int(len(xmls)*VAL_FRACTION))
val_xmls = xmls[:n_val]
train_xmls = xmls[n_val:]

pathlib.Path("/content/lists").mkdir(parents=True, exist_ok=True)
open("/content/lists/train.txt","w").write("\n".join(train_xmls))
open("/content/lists/val.txt","w").write("\n".join(val_xmls))

print("Train pages:", len(train_xmls), "| Val pages:", len(val_xmls))
print("First train:", train_xmls[:2])
print("First val:", val_xmls[:2])


## 5) Hyperparameters (+ auto device) — edit learning rates here

In [ ]:

# ======= EDIT ME (Hyperparameters) =======
import torch, shutil

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
print("Using device:", DEVICE)
KETOS = shutil.which("ketos") or "ketos"
print("ketos path:", KETOS)

# Recognition
REC_EPOCHS = 30
REC_BATCH  = 8
REC_OPTIM  = "Adam"    # or "SGD"
REC_LR     = 0.0001    # learning rate (decimal allowed)
REC_WD     = 1e-5

# Segmentation
SEG_EPOCHS = 20
SEG_BATCH  = 2
SEG_OPTIM  = "Adam"
SEG_LR     = 0.0005    # learning rate
# =========================================


## 6) Helpers — detect Kraken flags and run training with logs

In [ ]:

import subprocess, shlex

def detect_flags():
    # Detect whether CLI supports --lr or --lrate; --validation or --evaluation-files
    out = subprocess.run([KETOS, "train", "--help"], text=True, capture_output=True).stdout
    lr_flag   = "--lr" if "--lr" in out else "--lrate"
    val_flag  = "--validation" if "--validation" in out else "--evaluation-files"
    print(f"[detected] lr flag: {lr_flag}, validation flag: {val_flag}")
    return lr_flag, val_flag

LR_FLAG, VAL_FLAG = detect_flags()

def run_recognition(out_dir, epochs, batch, optim, lr, weight_decay, device, train_list, val_list):
    args = [
        KETOS, "train",
        "-f", FORMAT, f"@{train_list}",
        "-o", out_dir,
        "--device", device,
        "--epochs", str(epochs),
        "--batch-size", str(batch),
        "--optimizer", optim, LR_FLAG, str(lr), "--weight-decay", str(weight_decay),
        VAL_FLAG, f"@{val_list}",
    ]
    print(">>>", " ".join(shlex.quote(a) for a in args))
    # stream logs live
    return subprocess.run(args).returncode

def run_segmentation(out_dir, epochs, batch, optim, lr, device, train_list, val_list):
    args = [
        KETOS, "segtrain",
        "-f", FORMAT, f"@{train_list}",
        "-o", out_dir,
        "--device", device,
        "--epochs", str(epochs),
        "--batch-size", str(batch),
        "--optimizer", optim, LR_FLAG, str(lr),
        VAL_FLAG, f"@{val_list}",
    ]
    print(">>>", " ".join(shlex.quote(a) for a in args))
    return subprocess.run(args).returncode


## 7) 🔎 Smoke test — recognition (1 epoch, tiny batch)

In [ ]:

rc = run_recognition(
    out_dir="/content/models/rec_smoke",
    epochs=1, batch=2, optim=REC_OPTIM, lr=REC_LR, weight_decay=REC_WD,
    device="cpu",  # keep CPU for a fast sanity check
    train_list="/content/lists/train.txt",
    val_list="/content/lists/val.txt"
)
print("Return code:", rc)
assert rc == 0, "Smoke test failed — check the printed logs above."


## 8) ▶️ Recognition — full training

In [ ]:

rc = run_recognition(
    out_dir="/content/models/rec",
    epochs=REC_EPOCHS, batch=REC_BATCH, optim=REC_OPTIM, lr=REC_LR, weight_decay=REC_WD,
    device=DEVICE,
    train_list="/content/lists/train.txt",
    val_list="/content/lists/val.txt"
)
print("Return code:", rc)
assert rc == 0, "Recognition training failed."


## 9) ▶️ Segmentation — full training

In [ ]:

rc = run_segmentation(
    out_dir="/content/models/seg",
    epochs=SEG_EPOCHS, batch=SEG_BATCH, optim=SEG_OPTIM, lr=SEG_LR,
    device=DEVICE,
    train_list="/content/lists/train.txt",
    val_list="/content/lists/val.txt"
)
print("Return code:", rc)
assert rc == 0, "Segmentation training failed."


## 10) Evaluate recognition (CER/WER)

In [ ]:

import subprocess, shlex
args = ["ketos", "test", "-f", FORMAT, "/content/models/rec_best.mlmodel", "@/content/lists/val.txt"]
print(">>>", " ".join(shlex.quote(a) for a in args))
rc = subprocess.run(args).returncode
print("Return code:", rc)
assert rc == 0, "Evaluation failed."


## 11) Package models for download

In [ ]:

!mkdir -p /content/models && cd /content/models && ls -lh && zip -r ../trained_models.zip . && cd /content


## 12) Optional — Upload to msia.escriptorium.fr via API

In [ ]:

# UI upload (My Models → Upload) is simplest.
MSIA_URL   = "https://msia.escriptorium.fr"
API_TOKEN  = "PASTE_YOUR_TOKEN_HERE"   # keep secret or leave blank
MODEL_PATH = "/content/models/rec_best.mlmodel"   # or seg_best.mlmodel
MODEL_NAME = "rec_best"

if API_TOKEN and API_TOKEN != "PASTE_YOUR_TOKEN_HERE":
    import requests
    headers = {"Authorization": f"Token {API_TOKEN}"}
    files = {"file": (MODEL_NAME + ".mlmodel", open(MODEL_PATH, "rb"), "application/octet-stream")}
    data  = {"name": MODEL_NAME}
    resp = requests.post(f"{MSIA_URL}/api/models/", headers=headers, files=files, data=data)
    print("Status:", resp.status_code)
    try:
        print(resp.json())
    except Exception:
        print(resp.text[:800])
else:
    print("Skipping API upload. Use the UI or paste a valid API token.")
